# 🐐 searchgoat Test Notebook

This notebook verifies your searchgoat installation and Cribl connection.

**If you're seeing this, setup.sh worked!** Now let's test the connection.

---
## Step 1: Verify Imports

In [ ]:
import pandas as pd
import numpy as np
from searchgoat_jupyter import SearchClient, __version__

print(f"✓ searchgoat-jupyter v{__version__}")
print(f"✓ pandas v{pd.__version__}")
print(f"✓ numpy v{np.__version__}")

---
## Step 2: Verify Credentials

This reads from the `.env` file created by `setup.sh`.

In [ ]:
from searchgoat.config import CriblSettings

settings = CriblSettings()
print(f"✓ Client ID: {settings.client_id[:8]}...")
print(f"✓ Org ID: {settings.org_id}")
print(f"✓ Workspace: {settings.workspace}")
print(f"✓ API URL: {settings.api_base_url}")

---
## Step 3: Connect and Query

**⚠️ Edit the dataset name below** to match a dataset in your Cribl Search.

To find your datasets:
1. Go to Cribl Cloud → Search
2. Look at the Datasets panel on the left
3. Use one of those names below

In [ ]:
# ════════════════════════════════════════════════════════════
# EDIT THIS: Replace with your actual dataset name
# ════════════════════════════════════════════════════════════
DATASET = "YOUR_DATASET_NAME"  # e.g., "cribl_internal_logs", "main"
# ════════════════════════════════════════════════════════════

if DATASET == "YOUR_DATASET_NAME":
    print("⚠️  Please edit DATASET above to match a real dataset from your Cribl Search")
else:
    print(f"Dataset: {DATASET}")

In [ ]:
# Run the query
client = SearchClient()

df = client.query(
    f'cribl dataset="{DATASET}" | limit 10',
    earliest="-1h"
)

print(f"✓ Query successful!")
print(f"✓ Retrieved {len(df)} records")
print(f"✓ Columns: {list(df.columns)[:5]}{'...' if len(df.columns) > 5 else ''}")

---
## Step 4: Explore Results

In [ ]:
# View the data
df

In [ ]:
# Data types
df.dtypes

---
## Step 5: Test Job Workflow (Optional)

This shows the more granular job-based approach.

In [ ]:
# Submit job
job = client.submit(f'cribl dataset="{DATASET}" | limit 5', earliest="-1h")
print(f"Job ID: {job.id}")

# Wait for completion
job.wait()
print(f"Status: {job.status}")
print(f"Records: {job.record_count}")

# Get results
df2 = job.to_dataframe()
df2

---
## Step 6: Test Save/Load (Optional)

In [ ]:
# Save to parquet
job.save("test_output.parquet")
print("✓ Saved to test_output.parquet")

# Load it back
df_loaded = pd.read_parquet("test_output.parquet")
print(f"✓ Loaded {len(df_loaded)} records back")

# Cleanup
import os
os.remove("test_output.parquet")
print("✓ Cleaned up test file")

---
## 🎉 Success!

If you got here without errors, **searchgoat is working correctly**.

### Next Steps

- Write your own queries in new cells
- Use filters: `| where field="value"`
- Use aggregations: `| stats count() by field`
- Create new notebooks in this workspace folder

### Common Queries

```python
# Filter by field
df = client.query('cribl dataset="logs" | where level="ERROR" | limit 100', earliest="-24h")

# Aggregate
df = client.query('cribl dataset="logs" | stats count() by source', earliest="-24h")

# Longer time range, more results
df = client.query('cribl dataset="logs" | limit 10000', earliest="-7d", timeout=600)
```